In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense
from tensorflow.keras import callbacks
import gdown


class StockPricePredictor:
    def __init__(self, train_url, test_url, sequence_length=60):
        self.train_url = train_url
        self.test_url = test_url
        self.sequence_length = sequence_length
        self.scaler = MinMaxScaler(feature_range=(0, 1))
        self.model = None

    def download_data(self, train_output_file, test_output_file):
        gdown.download(self.train_url, train_output_file, quiet=False)
        gdown.download(self.test_url, test_output_file, quiet=False)
        print(f"Data downloaded: {train_output_file}, {test_output_file}")
        return train_output_file, test_output_file

    def load_and_prepare_data(self, train_file, test_file):
        # Load training data
        dataset_train = pd.read_csv(train_file)
        training_set = dataset_train.iloc[:, 1:2].values  # Extract 'Open' column
        
        # Scale training data
        training_set_scaled = self.scaler.fit_transform(training_set)
        
        # Create training sequences
        X_train, y_train = [], []
        for i in range(self.sequence_length, len(training_set_scaled)):
            X_train.append(training_set_scaled[i - self.sequence_length:i, 0])
            y_train.append(training_set_scaled[i, 0])
        
        X_train, y_train = np.array(X_train), np.array(y_train)
        X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
        
        # Load test data
        dataset_test = pd.read_csv(test_file)
        real_stock_price = dataset_test.iloc[:, 1:2].values  # Extract 'Open' column
        
        return X_train, y_train, dataset_train, dataset_test, real_stock_price

    def build_model(self):
        model = Sequential()
        model.add(LSTM(50, return_sequences=True, input_shape=(self.sequence_length, 1)))
        model.add(Dropout(0.2))
        model.add(LSTM(50, return_sequences=True))
        model.add(Dropout(0.2))
        model.add(LSTM(50))
        model.add(Dropout(0.2))
        model.add(Dense(1))
        model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])
        self.model = model
        print("Model built successfully!")
        return model

    def train_model(self, X_train, y_train, epochs=10, batch_size=32):
        checkpoint = callbacks.ModelCheckpoint("stock_price_lstm.keras", save_best_only=True)
        history = self.model.fit(
            X_train,
            y_train,
            epochs=epochs,
            batch_size=batch_size,
            callbacks=[checkpoint],
            verbose=1,
        )
        print("Model training completed!")
        return history

    def prepare_test_data(self, dataset_train, dataset_test):
        # Combine train and test datasets for creating test inputs
        dataset_total = pd.concat((dataset_train['Open'], dataset_test['Open']), axis=0)
        inputs_start_idx = max(0, len(dataset_total) - len(dataset_test) - self.sequence_length)
        inputs = dataset_total[inputs_start_idx:].values.reshape(-1, 1)
        inputs = self.scaler.transform(inputs)
        
        # Create test sequences
        X_test = []
        for i in range(self.sequence_length, len(inputs)):
            X_test.append(inputs[i - self.sequence_length:i, 0])
        
        X_test = np.array(X_test)
        X_test = X_test.reshape((X_test.shape[0], self.sequence_length, 1))
        return X_test

    def predict(self, X_test):
        predicted_stock_price = self.model.predict(X_test)
        predicted_stock_price = self.scaler.inverse_transform(predicted_stock_price)
        print("Prediction completed!")
        return predicted_stock_price

    def visualize_results(self, real_stock_price, predicted_stock_price):
        plt.figure(figsize=(10, 6))
        plt.plot(real_stock_price, color='red', label='Real Google Stock Price')
        plt.plot(predicted_stock_price, color='blue', label='Predicted Google Stock Price')
        plt.title('Google Stock Price Prediction')
        plt.xlabel('Time')
        plt.ylabel('Google Stock Price')
        plt.legend()
        plt.show()


# Main Execution
if __name__ == "__main__":
    # Google Drive file IDs
    train_file_id = "15FT8I1PFCoqunfNZYhKjZLNMOPKFputW"
    test_file_id = "1Zn1QlOlNFnwyySqyR3YpiWUrge3tWZ5Y"
    
    # Construct URLs
    train_url = f"https://drive.google.com/uc?id={train_file_id}"
    test_url = f"https://drive.google.com/uc?id={test_file_id}"
    
    # Initialize the predictor
    predictor = StockPricePredictor(train_url, test_url)
    
    # Download data
    train_file, test_file = predictor.download_data("Google_Stock_Price_Train.csv", "Google_Stock_Price_Test.csv")
    
    # Load and prepare data
    X_train, y_train, dataset_train, dataset_test, real_stock_price = predictor.load_and_prepare_data(train_file, test_file)
    
    # Build the model
    predictor.build_model()
    
    # Train the model
    predictor.train_model(X_train, y_train, epochs=10, batch_size=32)
    
    # Prepare test data
    X_test = predictor.prepare_test_data(dataset_train, dataset_test)
    
    # Make predictions
    predicted_stock_price = predictor.predict(X_test)
    
    # Visualize results
    predictor.visualize_results(real_stock_price, predicted_stock_price)
